In [1]:
using Plots
using QuadGK
using NLopt
using SpecialFunctions
using NLsolve
using Statistics
include("../../scaling_CGS.jl")

print_basic_info (generic function with 1 method)

In [2]:
function get_proton_radius(base, temp, B)
    v_thermal_p = sqrt(base.k_B * temperature_p / base.m_p)
    proton_gyro_freq = base.e * B / ( base.m_p * base.c)
    return v_thermal_p / proton_gyro_freq
end 

function get_plasma_frequency(base, n_p)
    return  base.e * sqrt(4 * π * n_p / base.m_e)
end

function get_mass_density_scale(base, n_p)
    mu = ( (base.m_e + base.m_p)  / 2. ) /  base.m_u
    mass_density = 2 * n_p * mu *  base.m_u
    mass_density_scale = mass_density 
    return mass_density_scale
end

function print_ds_info(scaling)
    println("")
    @printf("%-50s = %.4e \n","debye length", scaling.debye_len_code) 
    @printf("%-50s = %.4e %.4e %.4e \n","ds at n_cell=128, debye len per ds, dt=ds/c*0.25",  L_sim_x / nx[1], L_sim_x / nx[1] / scaling.debye_len_code, L_sim_x / nx[1] / scaling.c_code*0.25) 
    @printf("%-50s = %.4e %.4e %.4e \n","ds at n_cell=256, debye len per ds, dt=ds/c*0.25",  L_sim_x / nx[2], L_sim_x / nx[2] / scaling.debye_len_code, L_sim_x / nx[2] / scaling.c_code*0.25) 
    @printf("%-50s = %.4e %.4e %.4e \n","ds at n_cell=512, debye len per ds, dt=ds/c*0.25",  L_sim_x / nx[3], L_sim_x / nx[3] / scaling.debye_len_code, L_sim_x / nx[3] / scaling.c_code*0.25) 
    @printf("%-50s = %.4e %.4e %.4e \n","ds at n_cell=1024, debye len per ds, dt=ds/c*0.25", L_sim_x / nx[4], L_sim_x / nx[4] / scaling.debye_len_code, L_sim_x / nx[4] / scaling.c_code*0.25) 
end

print_ds_info (generic function with 1 method)

In [3]:
#---------- Values that are the same in all setups -------------------

#------------- real physical constants in CGS units ----------------
number_density=1e9 #* 0.5^2
B_flux= 25
temperature   = 1e6
temperature_e = temperature
temperature_p = temperature
#----------------------------------------------------------------------

mu_0_scaling = 1e0

#-----------------

L_sim_x = 25.6
L_sim_y = L_sim_x / 2
nx = [128,256,512,1024];

In [17]:
base = BaseUnits("CGS")
#------------- Scale base units ----------------
    #--------------- scaling parameters -------------------
electron_proton_ratio = base.m_p / base.m_e # realistic proton to electron mass
eps_0_scaling = 1e0
charge_scaling = 1e0
    #------ calculate electron mass scaing based on ratio of electron to proton mass -----
electron_mass_scaling = (base.m_p / base.m_e) / electron_proton_ratio
    #-------------------------------------------------------------------------------------
scale_base_units(base, eps_0_scaling, mu_0_scaling, electron_mass_scaling, charge_scaling)
#-----------------------------------------------


#---------- get plasma freq and proton radius in order to set length and time scales ---
ω_e = get_plasma_frequency(base, number_density)
r_p = get_proton_radius(base, temperature_p, B_flux)
#---------------------------------------------------------------------------------------

#------------------- set scaling units  ----------------------------------------
mass_density_scale = get_mass_density_scale(base, number_density)
length = r_p
length_scale = r_p
time_scale = 10 / (base.c / length)  #2 * pi / ω_e#2 * pi / ω_e
#---------------------------------------------------------------------------------------


scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale)
ds = 0.1
ppc = 128
set_macro_particle_weights(scaling, ds, ppc)
println(" speed of light in code units = ", scaling.c_code)
print_basic_info(scaling)
print_ds_info(scaling)
print_all_CGS(scaling)
                

 speed of light in code units = 10.0
lenght                 = 37.93936804601426
number_density         = 1.0e9
B                      = 25.0
T                      = 1.0e6

length_scale           = 37.93936804601426
mass_density_scale     = 1.6735328363560001e-15
time_scale             = 1.2655210974658426e-8
temperature_scale      = 1.0e6

e_mass_scale           = 1.0
charge_scaling         = 1.0
eps_0_scaling          = 1.0
mu_0_scaling           = 1.0

debye length                                       = 5.7520e-03 
ds at n_cell=128, debye len per ds, dt=ds/c*0.25   = 2.0000e-01 3.4771e+01 5.0000e-03 
ds at n_cell=256, debye len per ds, dt=ds/c*0.25   = 1.0000e-01 1.7385e+01 2.5000e-03 
ds at n_cell=512, debye len per ds, dt=ds/c*0.25   = 5.0000e-02 8.6927e+00 1.2500e-03 
ds at n_cell=1024, debye len per ds, dt=ds/c*0.25  = 2.5000e-02 4.3463e+00 6.2500e-04 

 Maxwell Lorents Factors used in code:
 k_E =                                             =  1.0000e+00 
 k_B =               

In [5]:
0.001 * scaling.c_code /  (scaling.B_flux_code * scaling.v_a_code )

0.8531153158877349

In [6]:
nx_all = [128,256,512,1024];
ds = 25.6 ./ nx_all
dt = ds ./ scaling.c_code * 0.25
ppc_all = [32,64,128]
n_species = 2
n_cores = 128
t_end = 300


number_of_cells = nx_all .* nx_all / 2
number_of_particles = number_of_cells * ppc_all' * n_species
time_per_timestep = 0.8e-6 * number_of_particles
time_per_sim_s = t_end ./ dt .* time_per_timestep / n_cores
time_per_sim_h = time_per_sim_s / 3600

using DataFrames

df = DataFrame(time_per_sim_h, :auto)
rename!(df, Symbol.("ppc_$(ppc)" for ppc in ppc_all))
# Add nx_all as a column to label the rows
df[!, :nx] = nx_all
# Set the :nx column as the first column
select!(df, :nx, Not(:nx))
# Print the DataFrame
println(df)

4×4 DataFrame
 Row │ nx     ppc_32      ppc_64     ppc_128    
     │ Int64  Float64     Float64    Float64    
─────┼──────────────────────────────────────────
   1 │   128   0.0546133   0.109227    0.218453
   2 │   256   0.436907    0.873813    1.74763
   3 │   512   3.49525     6.99051    13.981
   4 │  1024  27.962      55.9241    111.848


In [7]:
using DataFrames

df = DataFrame(time_per_sim_h, :auto)
rename!(df, Symbol.("ppc_$(ppc)" for ppc in ppc_all))
# Add nx_all as a column to label the rows
df[!, :nx] = nx_all
# Set the :nx column as the first column
select!(df, :nx, Not(:nx))
# Print the DataFrame
println(df)

4×4 DataFrame
 Row │ nx     ppc_32      ppc_64     ppc_128    
     │ Int64  Float64     Float64    Float64    
─────┼──────────────────────────────────────────
   1 │   128   0.0546133   0.109227    0.218453
   2 │   256   0.436907    0.873813    1.74763
   3 │   512   3.49525     6.99051    13.981
   4 │  1024  27.962      55.9241    111.848


In [8]:
n_timesteps = t_end ./ dt

4-element Vector{Float64}:
  60000.0
 120000.0
 240000.0
 480000.0

In [9]:
time_per_timestep  

4×3 Matrix{Float64}:
  0.41943   0.838861    1.67772
  1.67772   3.35544     6.71089
  6.71089  13.4218     26.8435
 26.8435   53.6871    107.374

In [10]:
10 / (base.c / length) 

1.2655210974658426e-8

In [11]:
base = BaseUnits("CGS")
#------------- Scale base units ----------------
    #--------------- scaling parameters -------------------
electron_proton_ratio = base.m_p / base.m_e # realistic proton to electron mass
#electron_proton_ratio = 400
eps_0_scaling = 1e2
charge_scaling = 1e-1
    #------ calculate electron mass scaing based on ratio of electron to proton mass -----
electron_mass_scaling = (base.m_p / base.m_e) / electron_proton_ratio
    #-------------------------------------------------------------------------------------
scale_base_units(base, eps_0_scaling, mu_0_scaling, electron_mass_scaling, charge_scaling)
#-----------------------------------------------


#---------- get plasma freq and proton radius in order to set length and time scales ---
ω_e = get_plasma_frequency(base, number_density)
r_p = get_proton_radius(base, temperature_p, B_flux)
#---------------------------------------------------------------------------------------

#------------------- set scaling units  ----------------------------------------
mass_density_scale = get_mass_density_scale(base, number_density)
length = r_p
length_scale = r_p
time_scale = 1 / (base.c / length)  #2 * pi / ω_e
#---------------------------------------------------------------------------------------


scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale)


ds = 0.1
ppc = 128
set_macro_particle_weights(scaling, ds, ppc)

println(" speed of light in code units = ", scaling.c_code)
print_basic_info(scaling)
print_ds_info(scaling)

print_all_CGS(scaling)

 speed of light in code units = 1.0
lenght                 = 37.93936804601425
number_density         = 1.0e9
B                      = 25.0
T                      = 1.0e6

length_scale           = 37.93936804601425
mass_density_scale     = 1.6735328363560001e-15
time_scale             = 1.2655210974658425e-8
temperature_scale      = 1.0e6

e_mass_scale           = 1.0
charge_scaling         = 0.1
eps_0_scaling          = 100.0
mu_0_scaling           = 1.0

debye length                                       = 5.7520e-02 
ds at n_cell=128, debye len per ds, dt=ds/c*0.25   = 2.0000e-01 3.4771e+00 5.0000e-02 
ds at n_cell=256, debye len per ds, dt=ds/c*0.25   = 1.0000e-01 1.7385e+00 2.5000e-02 
ds at n_cell=512, debye len per ds, dt=ds/c*0.25   = 5.0000e-02 8.6927e-01 1.2500e-02 
ds at n_cell=1024, debye len per ds, dt=ds/c*0.25  = 2.5000e-02 4.3463e-01 6.2500e-03 

 Maxwell Lorents Factors used in code:
 k_E =                                             =  1.0000e+00 
 k_B =              

In [22]:
37.93936804601425*5.7520e-02

2.1822724500067396

In [20]:
1.2655210974658425e-8* 200

2.531042194931685e-6

In [12]:
* 0.1 *  scaling.c_code /  (scaling.B_flux_code * scaling.v_a_code )

ErrorException: syntax: "*" is not a unary operator

In [13]:
nx_all = [128,256,512,1024];
ds = 25.6 ./ nx_all
dt = ds ./ scaling.c_code * 0.25
ppc_all = [32,64,128]
n_species = 2
n_cores = 128
t_end = 300


number_of_cells = nx_all .* nx_all / 2
number_of_particles = number_of_cells * ppc_all' * n_species
time_per_timestep = 0.8e-6 * number_of_particles
time_per_sim_s = t_end ./ dt .* time_per_timestep / n_cores
time_per_sim_h = time_per_sim_s / 3600

using DataFrames

df = DataFrame(time_per_sim_h, :auto)
rename!(df, Symbol.("ppc_$(ppc)" for ppc in ppc_all))
# Add nx_all as a column to label the rows
df[!, :nx] = nx_all
# Set the :nx column as the first column
select!(df, :nx, Not(:nx))
# Print the DataFrame
println(df)

4×4 DataFrame
 Row │ nx     ppc_32      ppc_64     ppc_128    
     │ Int64  Float64     Float64    Float64    
─────┼──────────────────────────────────────────
   1 │   128  0.00546133  0.0109227   0.0218453
   2 │   256  0.0436907   0.0873813   0.174763
   3 │   512  0.349525    0.699051    1.3981
   4 │  1024  2.7962      5.59241    11.1848


In [14]:
scaling.B_flux_code * scaling.v_a_code / scaling.c_code

0.01170260605639418